In [8]:
import pandas as pd
# Load the newly uploaded scaled data files for Pune, Nashik, Navi Mumbai, Nagpur, and Thane
pune_scaled_df = pd.read_excel(r'C:\Users\karan\Downloads\scaled\Scaled_Pune_Data.xlsx')
nashik_scaled_df = pd.read_excel(r'C:\Users\karan\Downloads\scaled\Scaled_Nashik_Data.xlsx')
navimumbai_scaled_df = pd.read_excel(r'C:\Users\karan\Downloads\scaled\Scaled_NaviMumbai_Data.xlsx')
nagpur_scaled_df = pd.read_excel(r'C:\Users\karan\Downloads\scaled\Scaled_Nagpur_Data.xlsx')
thane_scaled_df = pd.read_excel(r'C:\Users\karan\Downloads\scaled\Scaled_Thane_Data.xlsx')

# Display the first few rows to understand the structure of the data
pune_scaled_df.head(), nashik_scaled_df.head(), navimumbai_scaled_df.head(), nagpur_scaled_df.head(), thane_scaled_df.head()


(   Impressions     Reach     Likes  Love Reactions  Care Reactions  \
 0     0.844890  0.873089  0.320700        0.059524        0.105263   
 1     0.866866  0.768168  0.897959        0.000000        0.684211   
 2     0.149349  0.156649  0.029155        0.738095        0.842105   
 3     0.415618  0.271832  0.784257        0.250000        0.684211   
 4     0.009814  0.119372  0.571429        0.464286        0.368421   
 
    Haha Reactions  Wow Reactions  Sad Reactions  Angry Reactions  \
 0            0.00       1.000000          0.625             0.25   
 1            0.75       0.189189          0.875             0.75   
 2            0.05       0.527027          0.750             0.50   
 3            0.90       0.162162          0.750             0.25   
 4            0.05       0.567568          0.750             0.50   
 
    Comment Count  ...  Profile Clicks  Video Views  Hide Post  Report as Spam  \
 0          0.375  ...        0.467742     0.844350   0.000000            

In [26]:
import numpy as np
from scipy.optimize import minimize

# Pre-intervention data for control cities and treated unit (Pune)
pune_pre_scaled = pune_scaled_df.values  # Scaled data for Pune
control_cities = {
    "Thane": thane_scaled_df.values,
    "Nagpur": nagpur_scaled_df.values,
    "NaviMumbai": navimumbai_scaled_df.values,
    "Nashik": nashik_scaled_df.values
}s

# Function to calculate the difference between the treated unit and the synthetic control
def objective(weights, treated, controls):
    synthetic_control = np.zeros_like(treated)
    for i, weight in enumerate(weights):
        synthetic_control += weight * controls[i]
    # Calculate the sum of squared differences between the treated unit and synthetic control
    return np.sum((treated - synthetic_control) ** 2)

# Prepare the data for the objective function
treated_data = pune_pre_scaled  # Treated unit (Pune)
control_data = list(control_cities.values())  # Control units (Thane, Nagpur, Navi Mumbai, Nashik)

# Initial guess for weights (equal weights for each control city)
initial_weights = np.ones(len(control_data)) / len(control_data)

# Perform optimization to find the optimal weights
result = minimize(objective, initial_weights, args=(treated_data, control_data), 
                  bounds=[(0, 1)] * len(control_data), 
                  constraints={'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# Extract the optimal weights from the result
optimal_weights = result.x

# Construct the synthetic control using the optimal weights
synthetic_control = np.zeros_like(treated_data)
for i, weight in enumerate(optimal_weights):
    synthetic_control += weight * control_data[i]

# Display the optimal weights and synthetic control for inspection
optimal_weights, synthetic_control[:]  # Display the first 5 rows of synthetic control for inspection



synthetic_control_df = pd.DataFrame(synthetic_control, columns=pune_scaled_df.columns)

# Save the synthetic control data to an Excel file
synthetic_control_df.to_excel(r"C:\Users\karan\Downloads\Synthetic_Control_Pune.xlsx", index=False)


